In [1]:
import ast
import pandas as pd

pd.set_option('display.max_columns', None)

---

## COMENTARIOS:
* He utilizado la fecha `'31.12.2099'` tanto dentro de la variable `('aut_fijo', 'historico')` como en `('creadas', 'f_baja')` para indicar que no hay fecha baja. Tenía que poner una fecha para poder hacer comparaciones más fácilmente.
* Como veréis en el dataset hay muchas columnas que se pueden eliminar, unas porque tienen todas valores únicos (pj. `nsin`) y otras porque toman todas el mismo valor (pj. `c1`). He preferido pasaron todo el dataset en "bruto". Las únicas variables que he eliminado en el ETL han sido de carácter privado y anónimo como números de teléfono o dnis.

---

In [2]:
path = "MM_PredictivoFraude.csv"

In [3]:
df = pd.read_csv(path, dtype=str, header=[0, 1], index_col=0)
df.head(3)

casos_cc                                                             \
       nsin       ncaso pos        nexp          nepi tipo         ccc   
0  29899501  1300298995   1  1300298995  2019E0001248  ANL  0810971910   
1  29901901  1300299019   1  1300299019  2019E0001274   EC  0802448664   
2  29902501  1300299025   1  1300299025  2019E0001281   EC  0803609685   

                                                                              \
  regimen           regccc situacion es_atipico es_recaida     hay_at_previo   
0    0521  0521/0810971910   Cerrado      False      False  No hay AT previo   
1    0521  0521/0802448664   Cerrado      False      False  No hay AT previo   
2    0521  0521/0803609685   Cerrado      False      False  No hay AT previo   

                                                                           \
  duracion dur_previo dur_std   v_std     f_parte     f_graba      f_baja   
0        6        0.0    15.0     9.0  2019-01-04  2019-01-04  2019-01-02   
1      153        0.0    30.0  -123.0  2019-01-04  2019-01-04  2019-01-02   
2       30        0.0    15.0   -15.0  2019-01-04  2019-01-04  2019-01-03   

                                              \
   f_alta_adm  f_alta_med med_clave cie9_cod   
0  2019-01-07  2019-01-07       469    959.8   
1  2019-06-03  2019-06-03       856   726.90   
2  2019-02-01  2019-02-01       222     490.   

                                                               \
                                          cie9_descr dele_cod   
0  TRAUMISMO NO ESPECIF. OTROS SITIOS ESPECIFICAD...      108   
1                   ENTESOPATIA, LUGAR NO LOCALIZADO      108   
2  BRONQUITIS NO ESPECIFICADA COMO AGUDA NI COMO ...      117   

                                                                  \
               dele_nombre d_sem_baja mes_baja es_trafico gestor   
0  DELEGACION DE BARCELONA  MIERCOLES    ENERO      False   1042   
1  DELEGACION DE BARCELONA  MIERCOLES    ENERO      False   1042   
2       DELEGACION DE OLOT     JUEVES    ENERO      False   1021   

                                                                  \
  es_pl_empleado es_pl_activo es_pago_dir es_susp_ext f_susp_ext   
0          False        False       False       False        NaN   
1          False        False        True       False        NaN   
2          False        False        True       False        NaN   

                                                                           \
  es_derecho_cobro es_alta_prev_insp es_recaida_ip_denegada es_covid  mpq   
0             True             False                  False    False  NaN   
1             True             False                  False    False    M   
2             True             False                  False    False  NaN   

   casos_ep                                                             \
  es_atrium f_atrium prop_atrium f_resp_atrium resp_atrium    f_ini_pd   
0     False      NaN         NaN           NaN         NaN         NaN   
1     False      NaN         NaN           NaN         NaN  2019-01-05   
2     False      NaN         NaN           NaN         NaN  2019-01-06   

                             aut                                           \
  motivo_pd nrehab nbio registro ent_at sit_trab UG_TGSS sexo    FECHA_AT   
0       NaN    0.0  0.0       01    007       52  082020    M  2017-07-01   
1  1.2 SETA   16.0  0.0       01    007       01  081111    H  2019-01-01   
2  1.2 SETA    0.0  0.0       01    007       01  170404    H  2019-01-01   

                                                                      \
  OPCION_IT FECHA_OPCION MUTUA_IT FECHA_MUTUA_IT base_coti colectivo   
0      True   2017-07-01      007     2017-07-01   2000.69   0200069   
1      True   1996-01-01      007     2015-01-01    2077.8   0207780   
2      True   2003-02-01      007     2003-02-01    1500.0   0150000   

                                                                            \
  f_alta_o_ss 

In [ ]:
# df[('creadas', 'f_baja')] = df[('creadas', 'f_baja')].replace('2099-12-31', np.nan)


convert_dict = {
    ('casos_cc', 'duracion'): float, ('casos_cc', 'dur_previo'): float, ('casos_cc', 'dur_std'): float, ('casos_cc', 'v_std'): float,
    ('casos_ep', 'nrehab'): float, ('casos_ep', 'nbio'): float,
    ('aut', 'base_coti'): float, 
    ('creadas', 'edad'): float, ('creadas', 'naltas'): float, ('creadas', 'dias_en_alta'): float, ('creadas', 'next_fest'): float, ('creadas', 'past_fest'): float,
    ('creadas', 'nfest_14'): float, ('creadas', 'peso'): float

}
df = df.astype(convert_dict)

date_cols = [
    ('casos_cc', 'f_parte'), ('casos_cc', 'f_graba'), ('casos_cc', 'f_baja'), ('casos_cc', 'f_alta_adm'), ('casos_cc', 'f_alta_med'), ('casos_cc', 'f_susp_ext'),
    ('casos_ep', 'f_atrium'), ('casos_ep', 'f_resp_atrium'), ('casos_ep', 'f_ini_pd'), 
    ('aut', 'FECHA_AT'), ('aut', 'FECHA_OPCION'), ('aut', 'FECHA_MUTUA_IT'), ('aut', 'f_alta_o_ss'), ('aut', 'f_cese'), ('aut_fijo', 'f_nacm'),
    ('creadas', 'f_alta'), ('creadas', 'f_baja')

]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d').dt.date


df[('aut', 'HORA_PRE')] = pd.to_datetime(df[('aut', 'HORA_PRE')], format='%H:%M:%S').dt.time

In [ ]:
for i in range(len(df)):
    df.at[i, ('aut_fijo', 'historico')] = ast.literal_eval(df.at[i, ('aut_fijo', 'historico')])